# KASM on AWS EC2
> Guide to deploy KASM on AWS
toc: true
comments: true
layout: base

# KASM Information

KASM is relatively simple to install for such a powerful software. It allows us to use Docker based containers to complete actions on numerous platforms, from browsing the web on Chrome to running code on VS Code.

KASM allows anonymous containers to be accessed from any computer, which will be useful in allowing school chromebooks to run coding applications.

KASM is also highly customizable because of its usage of Docker containers. Docker containers can be used to run various applications if a user has created a container. Additionally, KASM can be integrated into browsers to safely open links and webistes. After a user is done with a running container, it will be stopped and there will be no evidence on the web that the container was ever open.

# Preparing an Instance

Before running KASM, there will be a few requirements to set up the AWS Instance. Log in to the AWS interface and select launch instance to begin.

## Minimum Requirements
The minimum system requirements for KASM are:
- 2 CPU Cores
- 4 GB Memory
- 50 GB Storage
- OS must be Ubuntu 22.04 or 20.04
<br>
These requirements are met by an AWS EC2 **t.2 medium server** and I will be using the OS setup of Ubuntu 22.04. Be sure to ask your teacher for permission before creating a t.2 medium server since these ones are not Free Tier Eligible.

![]({{ site.baseurl }}/images/t2setts.png)

## System Partition 

In order to create a KASM instance, there needs to be a partition created. We will create the partition right now. Connect to your console and run the commands below.

In [ ]:
free -h
# Shows Free storage on disk
lsblk
# Shows partitions and drives

In [ ]:
# Result of lsblk
NAME     MAJ:MIN RM   SIZE RO TYPE MOUNTPOINTS
loop0      7:0    0  24.4M  1 loop /snap/amazon-ssm-agent/6312
loop1      7:1    0  63.3M  1 loop /snap/core20/1852
loop2      7:2    0  55.6M  1 loop /snap/core18/2714
loop3      7:3    0 111.9M  1 loop /snap/lxd/24322
loop4      7:4    0  49.8M  1 loop /snap/snapd/18596
xvda     202:0    0     8G  0 disk 
├─xvda1  202:1    0   7.9G  0 part /
├─xvda14 202:14   0     4M  0 part 
└─xvda15 202:15   0   106M  0 part /boot/efi
# The xvda drive will be important for us
# We will be using a partition in xvda

Now, exit out of the console, and navigate to the **volumes tab** for the instance. We will need to modify the volume.

![]({{ site.baseurl }}/images/volumen.png)

Go to the modify tab and then increase the instance size. KASM needs *at least 50 GB of storage*, so I opted to change my storage to 60 GB from the original 8.

![]({{ site.baseurl }}/images/modifyv.png)

After that, we can exit the volumes tab and reconnect to the instance.

Since we changed the volume, there will be a few commands which we need to run in order to make sure that our instance will run with the change in volume.

In [ ]:
# Begin here

sudo parted -l
# Output will ask for a "Fix/Ignore?", input "fix"
# Will fix the instance volume after the change in volume

sudo fdisk /dev/xvda # Change xvda if that is not the disk you will be using
# This will open the fdisk utility for altering the drive, be careful with the commands entered

# Type each of these letters into fdisk
n # Creates new partition
# For settings, you can just enter through all of them if you would like, which will create a partition in the default name and of maximum size
p # Goes over the changes made
w # Writes the changes to the disk

sudo fdisk -l # shows the changes made to the drive
partprobe

# Steps to ensure that the swap will stay

sudo mkswap /dev/xvda2 # xvda2 is the swap partition, change to your own name if needed
# Copy the UUID listed after running the command
sudo swapon /dev/xvda2

sudo blkid # Will list out a lot of things, get the uuid of the partition which you created

# Now we will need to alter fstab to save the partition
sudo vi /etc/fstab

# Press "i" to insert in vi and create a new line to save the swap
# Follow this template to save the swap
UUID="d13a2905-ed9c-4ea8-b35f-4933ba12d1ed"     none    swap    sw      0 0 # Replace UUID
# Press "esc" to exit insert mode
# Enter ":wq!" to exit vi editor

The swap should be created, so now you can restart your instance in order to check the swap. Run commands like "free -h" and "lsblk" to confirm.

## Downloading KASM

Now, we can download KASM after our storage has been altered. Connect to your instance and run the following commands.

In [ ]:
sudo mkdir kasmdir # Makes new directory for kasm
cd kasmdir

sudo curl -O https://kasm-static-content.s3.amazonaws.com/kasm_release_1.13.0.002947.tar.gz # Will get the zip file for KASM
sudo tar -xf kasm_release_1.13.0.002947.tar.gz # Unzips the files
sudo bash kasm_release/install.sh -L 8084 # Runs the install script
# The "-L 8084" specifies a port for the install, which has a default port of 443

Save the login credentials which KASM outputs, as you will not be able to access them again. After that, KASM should be installed. However, there are a few steps we must take before we can access the site.

## Creating Outbound Rules

We will need to go to the Security group of the instance. We will need to add security instances for the ports.

Once clicking on the instance, click on the Security tab and go to the security group.

![]({{ site.baseurl }}/images/security.png)

Now, we have to have some ports which will be able to be accessed. The 22 port is already exposed, but I exposed port 80 with HTTP and the port I included in the install(8084) with a custom TCP.

![]({{ site.baseurl }}/images/addingport.png)

Now we will be able to access the instance. Go to [Public IP]:[Port] and there should be a KASM desktop waiting for you to log in.

# Things to Do on KASM

Now that KASM is installed, we have a containerized desktop which we can access from anywhere. We can download different containers in order to meet our needs, like Chrome or VS Code.

KASM is very customizable and there are many things which we can do with.

# Important Links

- [KASM Documentation](https://kasmweb.com/docs/latest/index.html)
- [Swap Partition on AWS](https://www.youtube.com/watch?v=oHW0quS4pV4&pp=ygUNYXdzIHBhcnRpdGlvbg%3D%3D)
- [NetworkChuck YouTube Video on KASM](https://www.youtube.com/watch?v=U7e-mcJdZok&t=920s&pp=ygURbmV0d29ya2NodWNrIGthc20%3D)